# Problem Set 3: Logistic Regression

## Exercise 1

Consider the following binary model
$P(Y_{i} = y_{i}) = \pi_{i}^{y_{i}}*(1-\pi_{i})^{1-y_{i}}, y_{i} \in \{0, 1\}$   
where  
$\pi_{i}(x_{i}, \beta) = \frac{e^{x'_{i}\beta}}{(1 + e^{x'_{i}\beta})} = \frac{1}{1+e^{-x'_{i}\beta}}$

1. Simulate this model with the probabilities as described above with the following values:

- $n =1000$
- $\beta_0=-2, \, \beta_1=0.1,\, \, \, \beta_2=1$.
- $x_{0i}=1 \, \forall \,i$ , $x_{1i}\sim \mathcal{U}(18,60), \, x_{2i}\sim \mathcal{B}(0.5)$. 

2. Estimate $\beta_0, \, \beta_1,\, \beta_2$ via maximum likelihood and calculate the standard errors.

$P(Y_{i} = 1) = \pi_{i}^{1}*(1-\pi_{i})^{1-1} = \pi_{i}$  
$\log(\frac{\pi_{i}}{1 - \pi_{i}}) = x'_{i}\beta$

In [ ]:
# Load necessary libraries
pacman::p_load(ggplot2, tidyverse, MASS, caret, ggpubr, maxLik)

In [ ]:
# Set.seed()
seed <- 40
set.seed(seed)

In [ ]:
# Parameters
n <- 1000
beta <- c(-2, 0.1, 1)
X1.min <- 18
X1.max <- 60
X2.P1 <- 0.5


In [ ]:
# Define Data Gnerating function
data.gen <- function(n, beta, X1.min, X1.max, X2.P1) {
  X0 <- rep(1, n)
  X1 <- sort(runif(n, X1.min, X1.max))
  X2 <- rbinom(n, 1, X2.P1)
  X <- cbind(X0, X1, X2)
  logodds <- X %*% beta
  pi_x <- 1 / (1 + exp(-logodds))
  y <- rbinom(n, 1, prob = pi_x)
  data <- cbind.data.frame(X, logodds, pi_x, y)
  return(data)
}

### a) Generate the data

In [ ]:
# a)
# Generate the training data
data.train <- data.gen(n, beta, X1.min, X1.max, X2.P1)
head(data.train)

X.train <- cbind(data.train$X0, data.train$X1, data.train$X2)
head(X.train)

# Generate the test data
data.test <- data.gen(n, beta, X1.min, X1.max, X2.P1) 
head(data.test)

X.test <- cbind(data.test$X0, data.test$X1, data.test$X2)
head(X.test)

In [ ]:
table(data.train$y)

In [ ]:
# Explore the data
# Scatterplot of ys on X1
ggplot(data.train, aes(x = X1, y = y, colour = factor(y))) + 
  geom_point()

In [ ]:
# ... ys on X2
ggplot(data.train, aes(x = X2, y = y, colour = factor(y))) + 
  geom_point()

In [ ]:
# Distribution of X1
ggplot(data.train, aes(X1)) + 
  geom_histogram()

In [ ]:
# Distribution of X2
ggplot(data.train, aes(X2)) + 
  geom_histogram() 


### b) Obtain the estimates


In [ ]:
# b)
# Estimate betas via ML (Logistic Regression)
Logit <- glm(y ~ X1 + X2, data = data.train, family = binomial)
summary(Logit)
(beta.logit <- Logit$coefficients)

#### Compared with estimates obtained from manually doing ML

$P(Y_{i} = y_{i}) = \pi_{i}^{y_{i}}*(1-\pi_{i})^{1-y_{i}}$  
$$
\begin{aligned}
L &= P(Y_{1} = y_{1},...Y_{n} = y_{n}) = \prod_{i = 1}^{n} \pi_{i}^{y_{i}}*(1-\pi_{i})^{1-y_{i}} \\
logL &= \sum_{i = 1}^{n} y_{i} * \log(\pi_{i}) + (1 - y_{i}) * \log(1 - \pi_{i})\\
\pi_{i} &= \frac{1}{1 + e^{-X\beta}}\\
1 - \pi_{i} &= \frac{1}{1 + e^{X\beta}}\\
logL &= \sum_{i = 1}^{n} y_{i}(-log(1 + e^{-X\beta}) + (1 - y_{i})(-log(1 + e ^ {X\beta}))\\
&= \sum_{i = 1}^{n} -y_{i}(log(1 + e^{-X\beta}) - (1 - y_{i})(log(1 + e ^ {X\beta}))
\end{aligned}
$$

In [ ]:
# Compared with manually doing ML

loglike<-function(b) # the likelihood function for the logit model
{
  ll <- sum(-data.train$y * log(1 + exp(-(X.train %*% b))) - 
              (1 - data.train$y) * log(1 + exp(X.train %*% b)))
  return(ll)
}

# Initialize estimation procedure
estim <- maxBFGS(loglike, finalHessian = TRUE, start = c(0, 0, 1)) 
beta.ML <- estim$estimate # give out parameter estimates
beta.ML
(beta.logit <- Logit$coefficients)

In [ ]:
# Standard Error of the Coefficients
estim.hess <- estim$hessian 
# the optimization routine returns the hessian matrix at the last iteration.
Cov <- -(solve(estim.hess))
# the covariance matrix is the (negative) inverse of the hessian matrix.
sde.ML <- sqrt(diag(Cov))#the standard errors are the square root of the diagonal of the inverse Hessian. 
sde.ML
stdEr(Logit)

### Prepare some data for plotting 
#### True probabilities and logodds

$P(Y_{i} = 1) = \pi_{i}^{1}*(1-\pi_{i})^{1-1} = \pi_{i} = \frac{1}{1+e^{-x'_{i}\beta}}$  
$\log(\frac{\pi_{i}}{1 - \pi_{i}}) = x'_{i}\beta$

In [ ]:
# Prepare some Data for plotting later
# "True" log odds and probs (never observe in real life)
head(data.train)
logodds.true.train <- data.train$logodds
probs.true.train <- data.train$pi_x

# "True" log odds and probs for test data
head(data.test)
logodds.true.test <- data.test$logodds 
probs.true.test <- data.test$pi_x


#### Fitted Log odds and Probailities, and Confidence Intervals

$\hat{P}(Y_{i} = 1) = \hat{\pi_{i}} = \frac{1}{1+e^{-x'_{i}\hat{\beta}}}$  
${\log}(\frac{\hat{\pi_{i}}}{1 - \hat{\pi_{i}}}) = x'_{i}\hat{\beta}$

In [ ]:
# Construct 95% CI for the estimated probs
# "Manual" way

beta.ML.lower <- beta.ML - 1.96 * sde.ML
beta.ML.upper <- beta.ML + 1.96 * sde.ML

logodds.ML.train <- X.train %*% beta.ML
logodds.ML.train.lower <- X.train %*% beta.ML.lower
logodds.ML.train.upper <- X.train %*% beta.ML.upper

probs.ML.train <- 1 / (1 + exp(-logodds.ML.train))
probs.ML.train.lower <- 1 / (1 + exp(-logodds.ML.train.lower))
probs.ML.train.upper <- 1 / (1 + exp(-logodds.ML.train.upper))

logodds.ML.test <- X.test %*% beta.ML
logodds.ML.test.lower <- X.test %*% beta.ML.lower
logodds.ML.test.upper <- X.test %*% beta.ML.upper

probs.ML.test <- 1 / (1 + exp(-logodds.ML.test))
probs.ML.test.lower <- 1 / (1 + exp(-logodds.ML.test.lower))
probs.ML.test.upper <- 1 / (1 + exp(-logodds.ML.test.upper))

In [ ]:
# Construct 95% CI for the estimated probs: "let-the-machine-do-it" way
# Fit the model again to the data.train
logit.train <- predict(Logit, data.train, se = T)

logodds.fit.train <- logit.train$fit
probs.fit.train <- 1 / (1 + exp(-logodds.fit.train))

logodds.lower.train <- logodds.fit.train - 1.96 * logit.train$se.fit # lower bound
logodds.upper.train <- logodds.fit.train + 1.96 * logit.train$se.fit # upper bound
lower.train <- 1 / (1 + exp(-logodds.lower.train)) 
upper.train <- 1 / (1 + exp(-logodds.upper.train))

# Marginal Effect of Pr(y = 1|X) = p(X) on X1
dx1.train <- beta.logit[2] * probs.fit.train * (1 - probs.fit.train)

In [ ]:
# Use the estimated betas to fit the test data
logit.test <- predict(Logit, newdata = data.test, se = T)
logodds.fit.test <- logit.test$fit

# Check if it's correct
head(logodds.fit.test)
head(X.test %*% beta.logit)

In [ ]:
# Construct 95% CI for the (test) estimated probs 
lower.test <- logodds.fit.test - 1.96 * logit.test$se.fit # lower bound of log odds
upper.test <- logodds.fit.test + 1.96 * logit.test$se.fit # upper bound...
probs.fit.test <- 1 / (1 + exp(-logodds.fit.test))
lower.test <- 1 / (1 + exp(-lower.test)) 
upper.test <- 1 / (1 + exp(-upper.test))

#### MSE, AVE and the Confusion Matrix

In [ ]:
# MSE and AVE
y.pred.train <-  c()
y.pred.test <- c()
threshold <- 0.65
for (i in 1:n) {
  if (probs.fit.train[i] >= threshold) {
    y.pred.train[i] <- 1
  }
  else {
    y.pred.train[i] <- 0
  }
  if (probs.fit.test[i] >= threshold) {
    y.pred.test[i] <- 1
  }
  else {
    y.pred.test[i] <- 0
  }
}
# Training error
(MSE <- sum(y.pred.train != data.train$y) / length(data.train$y))

# Testing error
(AVE <- sum(y.pred.test != data.test$y) / length(data.test$y))

#### For confusion matrices we factor() the ys for better interpretation

In [ ]:
# For Confusion Matrices we factor the ys
y.train.cfm <- factor(data.train$y, levels = c(0, 1), labels = c("Pos", "Neg"))
y.test.cfm <- factor(data.test$y, levels = c(0, 1), labels = c("Pos", "Neg"))
y.pred.train.cfm <- factor(y.pred.train, levels = c(0, 1), labels = c("Pos", "Neg"))
y.pred.test.cfm <- factor(y.pred.test, levels = c(0, 1), labels = c("Pos", "Neg"))

In [ ]:
(cfm.train <- table(y.pred.train.cfm, y.train.cfm, dnn = c("Predicted", "True")))
(addmargins(cfm.train))
addmargins(prop.table(cfm.train))
addmar

In [ ]:
(mosaic.train <- (mosaicplot(table(y.train.cfm, y.pred.train.cfm, dnn = c("True", "Predicted")))))


In [ ]:
(cfm.test <- table(y.pred.test.cfm, y.test.cfm, dnn = c("Predicted", "True")))
(addmargins(cfm.test))
addmargins(prop.table(cfm.test))
addmargins(prop.table(cfm.test, 2))


In [ ]:
(mosaic.test <- (mosaicplot(table(y.test.cfm, y.pred.test, dnn = c("True", "Predicted")))))

In [ ]:
MSE
0.102 + 0.057
AVE
0.077 + 0.062

### c) Interpretation

Consider the first observation. We'll calculate its log odds, odds and $P(Y_{1} = 1) = \pi_{1}$

$$
\begin{aligned}
\hat{log.odds_{1}} &= x'_{1}\hat{\beta} \\
\hat{odds_{1}} &= e^{\hat{log.odds_{1}}} \\
\hat{P}(Y_{1} = 1) &= \frac{1}{1 + e ^ {-\hat{log.odds_{1}}}} = \hat{\pi_{1}}
\end{aligned}
$$

In [ ]:
# c) Interpretation

# Logodss of the first observation and P(Y_1 = 1)
logit.train$fit[1] # "automatic"
X.train[1, ] %*% beta.logit # "manually"
# Odss of the first obs
(odds_1 <- exp(logit.train$fit[1]))
# With an odds of 1.76:1 the first observation is in group 1
# Or in other words, the probability that Y_1 = 1 is:
(pi_x_1 <- 1 / (1 + exp(-logit.train$fit[1]))) # P(Y_1 = 1) is around 64% 
                                              # gives an odds of ~ 64/36 ~ 1.7)

#### Marginal Effects
$\hat{log.odds_{1}} = x'_{1}\hat{\beta} = \hat{\beta_{0}} * x_{10} + \hat{\beta_{1}} * x_{11} + \hat{\beta_{2}} * x_{12}$  
One unit increase in $x_{11}$ increases $\hat{odds_{1}}$ by $e^{\hat{\beta_{1}}} - 1 \%$

In [ ]:
(exp(beta.logit[2]) - 1) * 100
logodds_1_new <- beta.logit[1] * X.train[1, 1] +
                    beta.logit[2] * (X.train[1, 2] + 1) +
                    beta.logit[3] * X.train[1, 3]
odds_1_new <- exp(logodds_1_new)
(odds_1_new - odds_1) / odds_1 * 100

What about $\frac{\partial \pi}{\partial X_1}?$
$$
\begin{aligned}
\pi_i&= \frac{1}{1 + e^{-(\beta_{0}x_{i0} + \beta_1x_{i1} + \beta_2x_{i2})}} \\
\frac{\partial \pi_{i}}{\partial x_{i1}} &= \frac{-1}{(1 + e^{-(\beta_{0}x_{i0} + \beta_1x_{i1} + \beta_2x_{i2})})^2}(-\beta_1)e^{\beta_{0}x_{i0} + \beta_1x_{i1} + \beta_2x_{i2})}\\
&= \beta_1\frac{1}{1 + e^{-(\beta_{0}x_{i0} + \beta_1x_{i1} + \beta_2x_{i2})}}\frac{e^{-(\beta_{0}x_{i0} + \beta_1x_{i1} + \beta_2x_{i2})}}{1 + e^{-(\beta_{0}x_{i0} + \beta_1x_{i1} + \beta_2x_{i2})}} \\
&= \beta_1\pi_i(1 - \pi_i)
\end{aligned}
$$
Consider again the first observation:  
$\frac{\partial \pi_{i}}{\partial x_{i1}}|x_{11} = \beta_1\pi_1(1 - \pi_1)$

In [ ]:
(dx1_1 <- beta.logit[2] * pi_x_1 * (1 - pi_x_1)) * 100 # dx1 of the 1st. obs: from math
pi_x_1_new <- 1 / (1 + exp(-logodds_1_new))
(pi_x_1_new - pi_x_1) * 100 # dx1 of obs 1: computational

One unit increase in x1 increases the predicted probability by 2.4 percentage points (from math) or 2.37 percentage points practically.

## d) VIsualizing the results

### Preparing data frame for plotting

In [ ]:
plt.data <- data.frame(X1.train = data.train$X1,
                       X1.test = data.test$X1,
                       X2.train = factor(data.train$X2),
                       X2.test = factor(data.test$X2),
                       y.train = data.train$y,
                       y.test = data.test$y,
                       probs.train = probs.fit.train,
                       probs.ML.train,
                       probs.test = probs.fit.test,
                       probs.ML.test,
                       probs.true.train,
                       probs.true.test,
                       upper.train,
                       probs.ML.train.upper,
                       lower.train,
                       probs.ML.train.lower,
                       upper.test,
                       probs.ML.test.upper,
                       lower.test,
                       probs.ML.test.lower,
                       dx1.train)

head(plt.data)  
str(plt.data)

In [ ]:
ggplot(plt.data, aes(X1.train)) +
  geom_line(aes(y = probs.fit.train, colour = X2.train), cex = 1.5)

In [ ]:
# Plot the marginal effect of X1
ggplot(plt.data, aes(X1.train)) +
  geom_line(aes(y = dx1.train, colour = X2.train), cex = 1.5)

In [ ]:
ggplot(plt.data, aes(X1.train)) +
  geom_line(aes(y = probs.fit.train, colour = X2.train), cex = 1.5) +
  geom_ribbon(aes(ymin = lower.train,
                  ymax = upper.train,
                  colour = X2.train),
                  alpha = 0.1)

In [ ]:
ggplot(plt.data, aes(X1.train)) +
  geom_line(aes(y = probs.fit.train, colour = X2.train), cex = 1) +
  geom_ribbon(aes(ymin = lower.train,
                  ymax = upper.train,
                  colour = X2.train),
                  alpha = 0.1) +
  geom_point(aes(y = y.train), size = 0.3) +
  geom_point(aes(y = probs.true.train), shape = "x", cex = 0.8)



In [ ]:
ggplot(plt.data, aes(X1.test)) +
geom_line(aes(y = probs.fit.test, colour = X2.test), cex = 1) 

In [ ]:
ggplot(plt.data, aes(X1.test)) +
geom_line(aes(y = probs.fit.test, colour = X2.test), cex = 1) +
geom_ribbon(aes(ymin = lower.test,
                ymax = upper.test,
                colour = X2.test),
            alpha = 0.1)

In [ ]:
ggplot(plt.data, aes(X1.test)) +
geom_line(aes(y = probs.fit.test, colour = X2.test), cex = 1) +
geom_ribbon(aes(ymin = lower.test,
                ymax = upper.test,
                colour = X2.test),
            alpha = 0.1) +
geom_point(aes(y = y.test), size = 0.3) +
geom_point(aes(y = probs.true.test), shape = "x", cex = 0.8)

In [ ]:
ggplot(plt.data, aes(X1.train)) +
  geom_line(aes(y = probs.ML.train, colour = X2.train), cex = 1) +
  geom_ribbon(aes(ymin = probs.ML.train.lower,
                  ymax = probs.ML.train.upper,
                  colour = X2.train),
              alpha = 0.1) +
  geom_point(aes(y = y.train), size = 0.3)

In [ ]:
logit.train.plt <- ggplot(plt.data, aes(X1.train)) +
  geom_line(aes(y = probs.fit.train, colour = X2.train), cex = 1) +
  geom_ribbon(aes(ymin = lower.train,
                  ymax = upper.train,
                  colour = X2.train),
              alpha = 0.1) +
  geom_point(aes(y = y.train), size = 0.3) 

ML.train.plt <- ggplot(plt.data, aes(X1.train)) +
  geom_line(aes(y = probs.ML.train, colour = X2.train), cex = 1) +
  geom_ribbon(aes(ymin = probs.ML.train.lower,
                  ymax = probs.ML.train.upper,
                  colour = X2.train),
              alpha = 0.1) +
  geom_point(aes(y = y.train), size = 0.3)

logit.test.plt <- ggplot(plt.data, aes(X1.test)) +
  geom_line(aes(y = probs.fit.test, colour = X2.test), cex = 1) +
  geom_ribbon(aes(ymin = lower.test,
                  ymax = upper.test,
                  colour = X2.test),
              alpha = 0.1) +
  geom_point(aes(y = y.test), size = 0.3)

ML.test.plt <- ggplot(plt.data, aes(X1.test)) +
  
  geom_line(aes(y = probs.ML.test, colour = X2.test), cex = 1) +
  geom_ribbon(aes(ymin = probs.ML.test.lower,
                  ymax = probs.ML.test.upper,
                  colour = X2.test),
              alpha = 0.1) +
  geom_point(aes(y = y.test), size = 0.3)

In [ ]:
ggarrange(logit.train.plt, ML.train.plt)

In [ ]:
ggarrange(logit.test.plt, ML.test.plt)

## Exercise 2: Simulations

In [ ]:
# Parameters
set.seed(seed)
nsim <- 100 # number of simulations
n <- 1000
beta <- beta
X1.min <- 18
X1.max <- 60
X2.P1 <- 0.5

In [ ]:
# Original data generating function
data.gen <- function(n, beta, X1.min, X1.max, X2.P1) {
  X0 <- rep(1, n)
  X1 <- sort(runif(n, X1.min, X1.max))
  X2 <- rbinom(n, 1, X2.P1)
  X <- cbind(X0, X1, X2)
  logodds <- X %*% beta
  pi_x <- 1 / (1 + exp(-logodds))
  y <- rbinom(n, 1, prob = pi_x)
  data <- cbind.data.frame(X, logodds, pi_x, y)
  return(data)
}

In [ ]:
# Define a function to get MSE and AVE from a model
geterr <- function(threshold) {
  # Generate data
  data.train <- data.gen(n, beta, X1.min, X1.max, X2.P1)
  data.test <- data.gen(n, beta, X1.min, X1.max, X2.P1) 
  X.train <- cbind(data.train$X0, data.train$X1, data.train$X2)
  X.test <- cbind(data.test$X0, data.test$X1, data.test$X2)
  
  # Get logistic estimates and fitted values
  Logit <- glm(y ~ X1 + X2, data = data.train, family = binomial)
  logit.train <- predict(Logit, data.train, se = T)
  logodds.fit.train <- logit.train$fit
  probs.fit.train <- 1 / (1 + exp(-logodds.fit.train))
  
  logit.test <- predict(Logit, newdata = data.test, se = T)
  logodds.fit.test <- logit.test$fit
  probs.fit.test <- 1 / (1 + exp(-logodds.fit.test))
  
  # MSE and AVE
  y.pred.train <-  c()
  y.pred.test <- c()
  
  for (i in 1:n) {
    if (probs.fit.train[i] >= threshold) {
      y.pred.train[i] <- 1
    }
    else {
      y.pred.train[i] <- 0
    }
    if (probs.fit.test[i] >= threshold) {
      y.pred.test[i] <- 1
    }
    else {
      y.pred.test[i] <- 0
    }
  }
  # Training error
  MSE <- sum(y.pred.train != data.train$y) / length(data.train$y)
  
  # Testing error
  AVE <- sum(y.pred.test != data.test$y) / length(data.test$y)
  errs <- cbind.data.frame(MSE, AVE)
  
  return(errs)
}

In [ ]:
# Time for simulations
# Simulate and store the interested values
errlist <- replicate(nsim, geterr(min(0.7)), simplify = F)
MSEs <- c()
AVEs <- c()
for (i in 1:nsim) {
  MSEs[i] <- errlist[[i]]$MSE
  AVEs[i] <- errlist[[i]]$AVE
}
MSEs
AVEs
plt.data.2 <- cbind.data.frame(sim = 1:nsim, err = MSEs, type = "MSEs") %>%
  rbind(data.frame(sim = 1:nsim, err = AVEs, type = "AVEs"))

In [ ]:
ggplot(plt.data.2, aes(sim, err, colour = type)) + 
  geom_point()

In [ ]:
ggplot(plt.data.2, aes(y = err, colour = type)) + 
  geom_boxplot()

In [ ]:
ggplot(plt.data.2, aes(x = err, colour = type)) + 
  geom_density()

In [ ]:
# Parameters
n <- 1000
beta <- beta
X1.min <- 18
X1.max <- 60
X2.P1 <- 0.5

In [ ]:
# Copy the data.gen function here for modifications
data.gen <- function(n, beta, X1.min, X1.max, X2.P1) {
  X0 <- rep(1, n)
  X1 <- sort(runif(n, X1.min, X1.max))
  X2 <- rbinom(n, 1, X2.P1)
  X <- cbind(X0, X1, X2)
  logodds <- X %*% beta
  pi_x <- 1 / (1 + exp(-logodds))
  y <- rep(0, n)
  for (i in 1:n) {
    if (pi_x[i] >= min(0.5)) {
      y[i] <- 1
    }
    else {
      y[i] <- 0
    }
  }
  data <- cbind.data.frame(X, logodds, pi_x, y)
  return(data)
}

In [ ]:
# Time for simulations
# Simulate and store the interested values
errlist <- replicate(nsim, geterr(min(0.5)), simplify = F)
MSEs <- c()
AVEs <- c()
for (i in 1:nsim) {
  MSEs[i] <- errlist[[i]]$MSE
  AVEs[i] <- errlist[[i]]$AVE
}
MSEs
AVEs
plt.data.2 <- cbind.data.frame(sim = 1:nsim, err = MSEs, type = "MSEs") %>%
  rbind(data.frame(sim = 1:nsim, err = AVEs, type = "AVEs"))

In [ ]:
ggplot(plt.data.2, aes(sim, err, colour = type)) + 
  geom_point()

In [ ]:
ggplot(plt.data.2, aes(y = err, colour = type)) + 
  geom_boxplot()

In [ ]:
ggplot(plt.data.2, aes(x = err, colour = type)) + 
  geom_density()